In [4]:
import os
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, create_sql_agent, AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.memory import ConversationBufferWindowMemory
from langchain.sql_database import SQLDatabase

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://qucy-openai-test.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["SERPAPI_API_KEY"] = os.getenv('SERPAPI_API_KEY')


# create large language model service
llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")

# f0c89ec0b7a44a0b978a9c832dd2c85c
# news_api_key

In [5]:
llm("Hey what's up ?")

'\n\nNot much, just hanging out. How about you?'

## 1. Use GPT+ Agent to query customer data (transaction data)

### Given below transaction table, we can use natural language to query below table

<img src="./credit_card_transaction_table.png" alt="Credit Card Transaction Table" width="1800"/>

In [10]:
db = SQLDatabase.from_uri("sqlite:////mnt/disk1/python-projects/LangChainDemo/demo.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# create agent
database_agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

# run agent
database_agent.run("What is my latest available credit limit ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I should check the schema of the credit_card_transactions table to see what columns it has.
Action: schema_sql_db
Action Input: credit_card_transactions
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	Groce

'Your latest available credit limit is 1150.0.'

In [11]:
database_agent.run("What was my total spending in 2023 April ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to look at the credit_card_transactions table.
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	Grocery Store	50.0000000000	purchase	12

'Your total spending in April 2023 was 550.0.'

In [12]:
database_agent.run("Can you show me a breakdown of my credit card by category ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to look at the schema of the credit_card_transactions table to see what columns I can use in my query
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-

'The breakdown of your credit card by category is: payment (200.0), disputed charge (150.0), suspicious activity (100.0), groceries (50.0), gas (30.0), dining out (20.0), rewards (0.0) and interest (0.0).'

In [13]:
database_agent.run("What were the fees or interest charged on my credit card on 2023 April ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to see the schema of the credit_card_transactions table to know what I can query for
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	G

'On 2023 April, the fees or interest charged on my credit card were 0.0 for the majority of the month, with the exception of the 15th, when the interest rate was 18.0.'

## 2. Use GPT+Agent to query latest financial market data (public stock price /other foreign exchange info in real-time)

## 3. Use GPT + Agent to query financial news